In [ ]:
#IMPORT PYTHON PACKAGES
import seaborn as sns
import pandas as pd
import scipy as sp
import scipy.integrate
import numpy  as np
import copy
import matplotlib.pylab as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter
import time
import datetime
import subprocess
import itertools
from itertools import groupby, repeat, islice
import math as math
import decimal as d
from decimal import *


#Plot settings
sns.set_style("whitegrid", rc={"axes.edgecolor": "k", "axes.linewidth":2.})

sns.set_style("ticks", {"xtick.major.size":8,"ytick.major.size":8})


sns.set_context("notebook",rc={"grid.linewidth": 0, 
                            "font.family":"Helvetica", "axes.labelsize":24.,"xtick.labelsize":24., 
                            "ytick.labelsize":24., "legend.fontsize":18.})

color_blind_safe = sns.color_palette("colorblind", 10) 

colors = sns.color_palette("tab10", 10) 


In [ ]:
filename = ['./profile_str_siE200','profile_str_siE400', 'profile_str_siE600', 'profile_str_siE800', 'profile_str_siE1000']

MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    #lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    #MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs
#MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity
MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_str_siE200'), np.loadtxt('Thermo_str_siE400'), np.loadtxt('Thermo_str_siE600'), np.loadtxt('Thermo_str_siE800'),
                       np.loadtxt('Thermo_str_siE1000')], dtype=object) 


labels = ['SiGeSi200','SiGeSi400', 'SiGeSi600', 'SiGeSi800', 'SiGeSi1000'] 

for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]


# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600], Temperature[1600:2400], Temperature[2400:3200], 
                         Temperature[3200:4000]], dtype=float)
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600], Coordinate[1600:2400], Coordinate[2400:3200], 
                       Coordinate[3200:4000]], dtype=float)


excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = [];temp_D_200Si = [];mid_temp_200Si = []; average_temp_200Si = [];
boundary_low_200Si = []; boundary_high_200Si = [];

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance

    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    
    slope2.append([T_fit_left[0], T_fit_right[0]])
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)    
    
    xhigh_left = dX[230:360]
    yhigh_left = dTemp[230:360]
   
    # Calculate coefficients
    left_stack = np.vstack([xhigh_left, np.ones(len(xhigh_left))]).T
    m, c = np.linalg.lstsq(left_stack, yhigh_left, rcond=None)[0]
    high_slope = m
    high_intercept = c

    xlow_left = dX[10:130]
    ylow_left = dTemp[10:130]
   
    # Calculate coefficients
    left_stack = np.vstack([xlow_left, np.ones(len(xlow_left))]).T
    m, c = np.linalg.lstsq(left_stack, ylow_left, rcond=None)[0]
    low_slope = m
    low_intercept = c
    
    temp_diff = high_intercept - low_intercept
    
    y_high_4 = (high_slope * dX[200]) + high_intercept
    y_low_4 = (low_slope * len(xlow_left/4.05)) + low_intercept
    temp_diff_4 = y_high_4 - y_low_4
    temp_D_200Si.append(temp_diff_4)

    x_axis_temp_loc = (1/4.05)*max(dX)*0.1
    values_tempD = np.linspace(y_low_4, y_high_4,10)
    count_tempD_height = len(values_tempD)

    mid_temp = sorted(dTemp)[len(dTemp) // 2]
    mid_temp_200Si.append(mid_temp)
    
    average_temp = np.mean(dTemp)
    average_temp_200Si.append(average_temp)
    
    boundary_high = dTemp[excluded_points1]
    boundary_high_200Si.append(boundary_high)
 
    boundary_low = dTemp[excluded_points1-27]
    boundary_low_200Si.append(boundary_low)
    
Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
LR_values_200 = []
ThermalsGe125 = []
hf_values = []
heat_fluxSi200Si_values = []

dt = 0.0005 # picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
    
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right1,Thermal_right2]
    Ttherm = ['%.15f' % elem for elem in Therm_val]
    ThermalsGe125.append(Ttherm)
    
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.15f' % elem for elem in TC_values]
    s_values.append(val)
    
    
    LR200 = [Thermal_left1, Thermal_left2, Thermal_right2, Thermal_right1]
    
    value = ['%.15f' % elem for elem in LR200]
    
    LR_values_200.append(value)

    flux_top_left = (Thermal_left2*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_top_right = (Thermal_right2*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    flux_bot_left = (Thermal_left1*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_bot_right = (Thermal_right1*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    
    fluxes = [flux_top_left,flux_top_right,flux_bot_left,flux_bot_right]
    heat_val = ['%.15f' % elem for elem in fluxes]
    
    heat_fluxSi200Si_values.append(heat_val)    
   
    hf_values.append(heat_val)
    sum_hf_values =  sum(fluxes)
    SUM_HF_200Si = []
    SUM_HF_200Si.append(sum_hf_values)
    
    
heat_flux_top_left200Si = [];heat_flux_top_right200Si = [];heat_flux_bot_left200Si = [];heat_flux_bot_right200Si = [];
for h in hf_values:
    heat_flux_top_left200Si.append(np.array(float(h[0]))) #flux_top_left
    heat_flux_top_right200Si.append(np.array(float(h[1]))) #flux_top_right
    heat_flux_bot_left200Si.append(np.array(float(h[2]))) #flux_bot_left
    heat_flux_bot_right200Si.append(np.array(float(h[3]))) #flux_bot_right


therm_averageSi200 = []
for item in s_values:
    therm_averageSi200.append(item[2])

therm_averageSi200 = []
for item in s_values:
    therm_averageSi200.append(np.array(float(item[2])))


av_silicon_SiGeSi_200 = (float(LR_values_200[0][0]) + float(LR_values_200[0][3]) +
               float(LR_values_200[1][0]) + float(LR_values_200[1][3]) +
               float(LR_values_200[2][0]) + float(LR_values_200[2][3]) +
               float(LR_values_200[3][0]) + float(LR_values_200[3][3]) +
               float(LR_values_200[4][0]) + float(LR_values_200[4][3])) / 10

av_silicon_SiGeSi_200_values = [float(LR_values_200[0][0]), float(LR_values_200[0][3]),
               float(LR_values_200[1][0]), float(LR_values_200[1][3]),
               float(LR_values_200[2][0]), float(LR_values_200[2][3]),
               float(LR_values_200[3][0]), float(LR_values_200[3][3]),
               float(LR_values_200[4][0]), float(LR_values_200[4][3])]


av_germanium_SiGeSi_200 = (float(LR_values_200[0][1]) + float(LR_values_200[0][2]) +
               float(LR_values_200[1][1]) + float(LR_values_200[1][2]) +
               float(LR_values_200[2][1]) + float(LR_values_200[2][2]) +
               float(LR_values_200[3][1]) + float(LR_values_200[3][2]) +
               float(LR_values_200[4][1]) + float(LR_values_200[4][2])) / 10

av_germanium_SiGeSi_200_values = [float(LR_values_200[0][1]), float(LR_values_200[0][2]),
               float(LR_values_200[1][1]), float(LR_values_200[1][2]),
               float(LR_values_200[2][1]), float(LR_values_200[2][2]),
               float(LR_values_200[3][1]), float(LR_values_200[3][2]),
               float(LR_values_200[4][1]), float(LR_values_200[4][2])]
 
getcontext().prec = 10
std_silicon_SiGeSi_200 = Decimal(np.std([float(LR_values_200[0][0]),float(LR_values_200[0][3]),
               float(LR_values_200[1][0]),float(LR_values_200[1][3]),
               float(LR_values_200[2][0]),float(LR_values_200[2][3]),
               float(LR_values_200[3][0]),float(LR_values_200[3][3]),
            float(LR_values_200[4][0]),float(LR_values_200[4][3])], dtype=np.float64))
                      

SE_silicon_SiGeSi_200 = std_silicon_SiGeSi_200/Decimal(math.sqrt(10))

getcontext().prec = 10
std_germanium_SiGeSi_200 = Decimal(np.std([float(LR_values_200[0][1]),float(LR_values_200[0][2]),
               float(LR_values_200[1][1]),float(LR_values_200[1][2]),
               float(LR_values_200[2][1]),float(LR_values_200[2][2]),
               float(LR_values_200[3][1]),float(LR_values_200[3][2]),
            float(LR_values_200[4][1]),float(LR_values_200[4][2])], dtype=np.float64))
                      

SE_germanium_SiGeSi_200 = std_germanium_SiGeSi_200/Decimal(math.sqrt(10))

sil = np.array([float(LR_values_200[0][0]),float(LR_values_200[0][3]),
               float(LR_values_200[1][0]),float(LR_values_200[1][3]),
               float(LR_values_200[2][0]),float(LR_values_200[2][3]),
               float(LR_values_200[3][0]),float(LR_values_200[3][3]),
               float(LR_values_200[4][0]),float(LR_values_200[4][3])])
ger = np.array([float(LR_values_200[0][1]),float(LR_values_200[0][2]),
               float(LR_values_200[1][1]),float(LR_values_200[1][2]),
               float(LR_values_200[2][1]),float(LR_values_200[2][2]),
               float(LR_values_200[3][1]),float(LR_values_200[3][2]),
               float(LR_values_200[4][1]),float(LR_values_200[4][2])])


sil_SiGeSi_200 = np.array([sil[0],sil[1]])
sil_SiGeSi_400 = np.array([sil[2],sil[3]])
sil_SiGeSi_600 = np.array([sil[4],sil[5]])
sil_SiGeSi_800 = np.array([sil[6],sil[7]])
sil_SiGeSi_1000 = np.array([sil[8],sil[9]])

ger_SiGeSi_200 = np.array([ger[0],ger[1]])
ger_SiGeSi_400 = np.array([ger[2],ger[3]])
ger_SiGeSi_600 = np.array([ger[4],ger[5]])
ger_SiGeSi_800 = np.array([ger[6],ger[7]])
ger_SiGeSi_1000 = np.array([ger[8],ger[9]])

In [ ]:
filename = ['./profile_str_geE200','profile_str_geE400', 'profile_str_geE600', 'profile_str_geE800', 'profile_str_geE1000']

MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    #lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    #MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs
#MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity
MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_str_geE200'), np.loadtxt('Thermo_str_geE400'), np.loadtxt('Thermo_str_geE600'), np.loadtxt('Thermo_str_geE800'),
                       np.loadtxt('Thermo_str_geE1000')], dtype=object) 


labels = ['GeSiGe200','GeSiGe400', 'GeSiGe600', 'GeSiGe800', 'GeSiGe1000'] 

Sim_no_dataset = [1,1,1,1]
Time_no_dataset = [6000000,6000000,6000000,6000000]
Interval_no_dataset = [0.5e-3,0.5e-3,0.5e-3,0.5e-3] # picoseconds
mxc_no_dataset = [10,10,10,10] 

for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]

# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600], Temperature[1600:2400], Temperature[2400:3200], 
                         Temperature[3200:4000]], dtype=float)
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600], Coordinate[1600:2400], Coordinate[2400:3200], 
                       Coordinate[3200:4000]], dtype=float)


excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 120
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = [];temp_D_200Ge = [];mid_temp_200Ge = []; average_temp_200Ge = [];
boundary_low_200Ge = []; boundary_high_200Ge = [];

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
    
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    
    slope2.append([T_fit_left[0], T_fit_right[0]])

    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
    
    xhigh_left = dX[230:390]
    yhigh_left = dTemp[230:390]
   
    # Calculate coefficients
    left_stack = np.vstack([xhigh_left, np.ones(len(xhigh_left))]).T
    m, c = np.linalg.lstsq(left_stack, yhigh_left, rcond=None)[0]
    high_slope = m
    high_intercept = c
    
    xlow_left = dX[10:170]
    ylow_left = dTemp[10:170]
  
    # Calculate coefficients
    left_stack = np.vstack([xlow_left, np.ones(len(xlow_left))]).T
    m, c = np.linalg.lstsq(left_stack, ylow_left, rcond=None)[0]
    low_slope = m
    low_intercept = c
    
    temp_diff = high_intercept - low_intercept
      
    y_high_4 = (high_slope * dX[205]) + high_intercept
    y_low_4 = (low_slope * len(xlow_left/3.9)) + low_intercept
    temp_diff_4 = y_high_4 - y_low_4
    temp_D_200Ge.append(temp_diff_4)

    x_axis_temp_loc = (1/3.9)*max(dX)*0.1
    values_tempD = np.linspace(y_low_4, y_high_4,10)
    count_tempD_height = len(values_tempD)

    mid_temp = sorted(dTemp)[len(dTemp) // 2]
    mid_temp_200Ge.append(mid_temp)
    
    average_temp = np.mean(dTemp)
    average_temp_200Ge.append(average_temp)

    boundary_high = dTemp[excluded_points1]
    boundary_high_200Ge.append(boundary_high)
 
    boundary_low = dTemp[excluded_points1-27]
    boundary_low_200Ge.append(boundary_low)

Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
LR_values_200 = []
ThermalsGe125 = []
hf_values = [];heat_flux200Ge_values = []
dt = 0.0005 # picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
    
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right1,Thermal_right2]
    Ttherm = ['%.15f' % elem for elem in Therm_val]
    ThermalsGe125.append(Ttherm)
    
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.15f' % elem for elem in TC_values]
    s_values.append(val)
    
    
    LR200 = [Thermal_left1, Thermal_left2, Thermal_right2, Thermal_right1]
    value = ['%.15f' % elem for elem in LR200]
    LR_values_200.append(value)

    df = Slope2[simNo,0]*1e10
    

    flux_top_left = (Thermal_left2*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_top_right = (Thermal_right2*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    flux_bot_left = (Thermal_left1*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_bot_right = (Thermal_right1*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    
    fluxes = [flux_top_left,flux_top_right,flux_bot_left,flux_bot_right]
    heat_val = ['%.15f' % elem for elem in fluxes]
    
    heat_flux200Ge_values.append(heat_val)
   
    hf_values.append(heat_val)
    sum_hf_values =  sum(fluxes)
    SUM_HF_200Ge = []
    SUM_HF_200Ge.append(sum_hf_values)

    
    
heat_flux_top_left200Ge = [];heat_flux_top_right200Ge = [];heat_flux_bot_left200Ge = [];heat_flux_bot_right200Ge = [];
for h in hf_values:
    heat_flux_top_left200Ge.append(np.array(float(h[0]))) #flux_top_left
    #heat_flux_top_right200Ge.append(np.array(float(h[1]))) flux_top_right
    #heat_flux_bot_left200Ge.append(np.array(float(h[2]))) flux_bot_left
    #heat_flux_bot_right200Ge.append(np.array(float(h[3]))) flux_bot_right





therm_averageGe200 = []
for item in s_values:
    therm_averageGe200.append(item[2])

therm_averageGe200 = []
for item in s_values:
    therm_averageGe200.append(np.array(float(item[2])))



av_germanium_GeSiGe_200 = (float(LR_values_200[0][0]) + float(LR_values_200[0][3]) +
               float(LR_values_200[1][0]) + float(LR_values_200[1][3]) +
               float(LR_values_200[2][0]) + float(LR_values_200[2][3]) +
               float(LR_values_200[3][0]) + float(LR_values_200[3][3]) +
            float(LR_values_200[4][0]) + float(LR_values_200[4][3])) / 10

av_germanium_GeSiGe_200_values = [float(LR_values_200[0][0]), float(LR_values_200[0][3]),
               float(LR_values_200[1][0]), float(LR_values_200[1][3]),
               float(LR_values_200[2][0]), float(LR_values_200[2][3]),
               float(LR_values_200[3][0]), float(LR_values_200[3][3]),
            float(LR_values_200[4][0]), float(LR_values_200[4][3])]



av_silicon_GeSiGe_200 = (float(LR_values_200[0][1]) + float(LR_values_200[0][2]) +
               float(LR_values_200[1][1]) + float(LR_values_200[1][2]) +
               float(LR_values_200[2][1]) + float(LR_values_200[2][2]) +
               float(LR_values_200[3][1]) + float(LR_values_200[3][2]) +
            float(LR_values_200[4][1]) + float(LR_values_200[4][2])) / 10

av_silicon_GeSiGe_200_values = [float(LR_values_200[0][1]), float(LR_values_200[0][2]),
               float(LR_values_200[1][1]), float(LR_values_200[1][2]),
               float(LR_values_200[2][1]), float(LR_values_200[2][2]),
               float(LR_values_200[3][1]), float(LR_values_200[3][2]),
            float(LR_values_200[4][1]), float(LR_values_200[4][2])]

getcontext().prec = 10
std_germanium_GeSiGe_200 = Decimal(np.std([float(LR_values_200[0][0]),float(LR_values_200[0][3]),
               float(LR_values_200[1][0]),float(LR_values_200[1][3]),
               float(LR_values_200[2][0]),float(LR_values_200[2][3]),
               float(LR_values_200[3][0]),float(LR_values_200[3][3]),
            float(LR_values_200[4][0]),float(LR_values_200[4][3])], dtype=np.float64))
                      

SE_germanium_GeSiGe_200 = std_germanium_GeSiGe_200/Decimal(math.sqrt(10))

getcontext().prec = 10
std_silicon_GeSiGe_200 = Decimal(np.std([float(LR_values_200[0][1]),float(LR_values_200[0][2]),
               float(LR_values_200[1][1]),float(LR_values_200[1][2]),
               float(LR_values_200[2][1]),float(LR_values_200[2][2]),
               float(LR_values_200[3][1]),float(LR_values_200[3][2]),
            float(LR_values_200[4][1]),float(LR_values_200[4][2])], dtype=np.float64))
                      

SE_silicon_GeSiGe_200 = std_silicon_GeSiGe_200/Decimal(math.sqrt(10))

ger = np.array([float(LR_values_200[0][0]),float(LR_values_200[0][3]),
               float(LR_values_200[1][0]),float(LR_values_200[1][3]),
               float(LR_values_200[2][0]),float(LR_values_200[2][3]),
               float(LR_values_200[3][0]),float(LR_values_200[3][3]),
               float(LR_values_200[4][0]),float(LR_values_200[4][3])])
sil = np.array([float(LR_values_200[0][1]),float(LR_values_200[0][2]),
               float(LR_values_200[1][1]),float(LR_values_200[1][2]),
               float(LR_values_200[2][1]),float(LR_values_200[2][2]),
               float(LR_values_200[3][1]),float(LR_values_200[3][2]),
               float(LR_values_200[4][1]),float(LR_values_200[4][2])])

ger_GeSiGe_200 = np.array([ger[0],ger[1]])
ger_GeSiGe_400 = np.array([ger[2],ger[3]])
ger_GeSiGe_600 = np.array([ger[4],ger[5]])
ger_GeSiGe_800 = np.array([ger[6],ger[7]])
ger_GeSiGe_1000 = np.array([ger[8],ger[9]])

sil_GeSiGe_200 = np.array([sil[0],sil[1]])
sil_GeSiGe_400 = np.array([sil[2],sil[3]])
sil_GeSiGe_600 = np.array([sil[4],sil[5]])
sil_GeSiGe_800 = np.array([sil[6],sil[7]])
sil_GeSiGe_1000 = np.array([sil[8],sil[9]])

In [ ]:
#R_int = delta T/J
#J = delta KE/Ac.delta t
#delta t = 0.5 fs
A_t_nm = 0.1
A_t_m = 1e-10


radius_sq = ((54.3*A_t_m)/2)**2
print(((54.3*A_t_m)/2))
print(54.3*A_t_m)

cross_section = np.pi*radius_sq

d_t = 3e-9

#Ac_dt = 2*cross_section * d_t
Ac_dt = 2* d_t


In [ ]:
#SiGeSi

R_int_str_s200 = temp_D_200Si[0]/np.sum((float(heat_fluxSi200Si_values[0][0]),float(heat_fluxSi200Si_values[0][2])), dtype=np.float32)

R_int_str_s400 = temp_D_200Si[1]/np.sum((float(heat_fluxSi200Si_values[1][0]),float(heat_fluxSi200Si_values[1][2])), dtype=np.float32)

R_int_str_s600 = temp_D_200Si[2]/np.sum((float(heat_fluxSi200Si_values[2][0]),float(heat_fluxSi200Si_values[2][2])), dtype=np.float32)

R_int_str_s800 = temp_D_200Si[3]/np.sum((float(heat_fluxSi200Si_values[3][0]),float(heat_fluxSi200Si_values[3][2])), dtype=np.float32)

R_int_str_s1000 = temp_D_200Si[4]/np.sum((float(heat_fluxSi200Si_values[4][0]),float(heat_fluxSi200Si_values[4][2])), dtype=np.float32)



In [ ]:
#GeSiGe

R_int_str_g200 = temp_D_200Si[0]/np.sum((float(heat_fluxSi200Si_values[0][0]),float(heat_fluxSi200Si_values[0][2])), dtype=np.float32)

R_int_str_g400 = temp_D_200Si[1]/np.sum((float(heat_fluxSi200Si_values[1][0]),float(heat_fluxSi200Si_values[1][2])), dtype=np.float32)

R_int_str_g600 = temp_D_200Si[2]/np.sum((float(heat_fluxSi200Si_values[2][0]),float(heat_fluxSi200Si_values[2][2])), dtype=np.float32)

R_int_str_g800 = temp_D_200Si[3]/np.sum((float(heat_fluxSi200Si_values[3][0]),float(heat_fluxSi200Si_values[3][2])), dtype=np.float32)

R_int_str_g1000 = temp_D_200Si[4]/np.sum((float(heat_fluxSi200Si_values[4][0]),float(heat_fluxSi200Si_values[4][2])), dtype=np.float32)
